In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!pip install ultralytics -q
!pip install tqdm -q

In [10]:
import sys
sys.path.append('/content/drive/Othercomputers/Desktop/src')

In [11]:
import logging
from pathlib import Path
import argparse

from tqdm import tqdm

from Preprocessor import Preprocessor
from YOLODetector import YOLODetector
from YOLOTrainer import YOLOTrainer
from config import (
    CLIPS_DIR,
    CVAT_DIR,
    OUTPUT_DIR,
    YOLO_MODEL,
    YOLO_CONFIDENCE_THRESHOLD,
    YOLO_BATCH_SIZE,
)
from visualize import create_detection_video

In [12]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# logger.info("Starting YOLO fine-tuning")
# trainer = YOLOTrainer(
#     model_path=YOLO_MODEL,
#     clips_dir=CLIPS_DIR,
#     cvat_dir=CVAT_DIR,
#     output_dir=OUTPUT_DIR
# )

# try:
#     trainer.train()
#     trainer.export_model()
# finally:
#     trainer.cleanup()

In [15]:
logger.info("Loading fine-tuned model for detection")
detector = YOLODetector(
    model_name=str(Path(OUTPUT_DIR) / 'finetune' / 'weights' / 'best.pt'),
    conf_threshold=YOLO_CONFIDENCE_THRESHOLD
)

logger.info("Processing clips")
preprocessor = Preprocessor(CLIPS_DIR, CVAT_DIR)

for clip in preprocessor.get_clips():
    logger.info(f"Processing clip {clip.video_id}")

    output_dir = Path(OUTPUT_DIR) / 'detections' / clip.video_id
    output_dir.mkdir(parents=True, exist_ok=True)

    metadata = preprocessor.get_clip_metadata(clip)
    frame_count = metadata['frame_count']
    fps = metadata['fps']

    frames = []
    detections = []

    with tqdm(total=frame_count, desc="Processing frames") as pbar:
        for frame_indices, batch_frames in preprocessor.process_clip_frames(
                clip,
                lambda x: x,
                batch_size=YOLO_BATCH_SIZE
        ):
            batch_detections = detector.detect_video(
                batch_frames,
                batch_size=YOLO_BATCH_SIZE
            )

            frames.extend(batch_frames)
            detections.extend(batch_detections)

            batch_output_path = output_dir / f'batch_{frame_indices[0]:06d}.mp4'
            create_detection_video(
                frames=batch_frames,
                detections=batch_detections,
                output_path=str(batch_output_path),
                fps=fps
            )

            pbar.update(len(frame_indices))

            frames = []
            detections = []

    logger.info(f"Finished processing clip {clip.video_id}")

logger.info("Done!")

Processing frames:  98%|█████████▊| 6400/6551 [01:39<00:02, 64.50it/s]


KeyboardInterrupt: 